### Importacao de bibliotecas

In [32]:
import pandas as pd
import boto3

### Carregamento dos dados

In [2]:
s3 = boto3.client('s3')

In [3]:
obj = s3.get_object(Bucket="arquivos-equipe04", Key="target_cohort.vcf.gz")

rows = []
for line in obj["Body"].iter_lines():
    line = line.decode("utf-8").split('\t')
    rows.append(line)

In [4]:
vcf = pd.DataFrame(rows)

In [5]:
vcf = vcf.dropna()
vcf.columns = vcf.loc[3397]
vcf = vcf.drop(vcf.index[0])

In [7]:
for col in [x for x in vcf.columns if x.startswith('Sample')]:
    vcf[col] = [line[0] for line in vcf[col].str.split(':')]

In [9]:
vcf.to_csv('target_cohort_processed.csv')

In [10]:
df38 = pd.read_table('s3://arquivos-equipe04/PGS002296_hmPOS_GRCh38.tsv')

In [11]:
df37 = pd.read_csv('s3://arquivos-equipe04/PGS002296_hmPOS_GRCh37.txt', sep='\t', skiprows=19)

In [12]:
functional = pd.read_table('s3://arquivos-equipe04/snps-functional-classification.tsv')

In [13]:
phenotype = pd.read_table('s3://arquivos-equipe04/datathon-pheno.tsv')

In [33]:
snps =  pd.read_table('s3://arquivos-equipe04/ld-snps.tsv')

In [ ]:
vcf.reset_index(drop=True,inplace=True)

### Calculo do PRS para populacao brasileira

#### Funcoes 

In [34]:
def load_effect_sizes(effect_sizes):
    """Get effect size for each variant"""
    # Example: load_effect_sizes("resources/prs-scores-PGS002296.tsv")
    effect_sizes = effect_sizes[['chr', 'pos_hg38', 'effect_allele', 'effect_weight']]
    effect_sizes['chr'] = effect_sizes['chr'].astype(str)
    effect_sizes['pos_hg38'] = effect_sizes['pos_hg38'].astype(int)
    effect_sizes['locus'] = effect_sizes['chr'].astype(str) + ':' + effect_sizes['pos_hg38'].astype(str) # pylint: disable=line-too-long
    effect_sizes_dict = effect_sizes[['locus', 'effect_weight', 'effect_allele']].set_index('locus').T.to_dict() # pylint: disable=line-too-long

    return effect_sizes_dict

In [35]:
def calc_vcf_prs(vcf_filename: str, risk_alleles_weight: dict) -> dict:
    """
    Calculate PRS of each sample by multiplying the effect_size (weight) by the number of risk alleles
    
    Args:
        vcf_filename (str): path to the VCF file;
        risk_alleles_weight (dict): dictionary as follows
        {'chr1:959139': {'weight': 0.05, 'risk_allele': 'G'},
        'chr1:1127258': {'weight': -0.016, 'risk_allele': 'C'},
        'chr1:1748780': {'weight': 0.021, 'risk_allele': 'G'},
        'chr1:2115499': {'weight': -0.019, 'risk_allele': 'G'}} 
    
    Returns:
        dict: as follows
        {'Sample_01': 2.3,
        'Sample_02': -4.5,
        'Sample_03': -2.8,
        'Sample_04': 3.6,
        'Sample_05': 5.17
        }   
    """
    

    got_header = False
    prs_sum = []
    with open(vcf_filename, 'r') as vcf_file:
        for raw_line in vcf_file:
            line = raw_line.split('\t')

            # Skip commentaries
            if raw_line[0:2] == '##':
                continue

            if not got_header:
                # get headers
                vcf_names = [x.strip() for x in line]
                vcf_names[0] = 'CHROM'


                index_chr = vcf_names.index('CHROM')
                index_pos = vcf_names.index('POS')

                got_header = True

            else:
                prs_sum = process_line(
                    line, vcf_names, risk_alleles_weight,
                    index_chr, index_pos, prs_sum)


    sample_names = vcf_names[9::]
    return dict(zip(sample_names, [ round(x, 5) for x in prs_sum ]))

def process_line(line, vcf_names, risk_alleles_weight, index_chr, index_pos, prs_sum): # pylint: disable=line-too-long
    """Calculate PRS sum for all samples"""
    locus = line[index_chr]+":"+line[index_pos]

    try:
        risk_variant_locus = risk_alleles_weight[locus]['weight']
        risk_allele = risk_alleles_weight[locus]['risk_allele']

        # get risk_allele index
        risk_allele_index = get_effect_risk_index(line, risk_allele, vcf_names)

        n_samples = len(vcf_names)

        sample_effects = [
            sum_risk_allele_index(
                sample_genotype = x.split(":")[0],
                risk_allele_index = risk_allele_index,
                risk_variant_locus = risk_variant_locus) for x in line[9:n_samples]
        ]

        return add_sample_effect(sample_effects, prs_sum)

    except KeyError:
        print(f"Variant {locus} present in VCF is not present in PRS file")

        return prs_sum

def add_sample_effect(values_to_add, prs_sum):
    """Create sum of PRS of each sample"""
    if prs_sum == []:
        new_sum = values_to_add
    else:
        new_sum = [x + y for x, y in zip(values_to_add, prs_sum)]

    return new_sum


def sum_risk_allele_index(sample_genotype, risk_allele_index, risk_variant_locus):
    """Calc additive model"""
    return sample_genotype.count(str(risk_allele_index)) * risk_variant_locus

def get_effect_risk_index(line, risk_allele, vcf_names):
    """Get GT index for risk allele"""
    try:
        risk_allele_index = line[vcf_names.index('ALT')].split(",").index(risk_allele) + 1
    except ValueError:
        risk_allele_index = -1

    # if risk allele is are the reference allele
    if line[vcf_names.index('REF')] == risk_allele:
        risk_allele_index =  0

    return risk_allele_index

#### Calculo de efeitos por variante

In [16]:
effect_size = load_effect_sizes(df38)

/tmp/ipykernel_13117/2241656604.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effect_sizes['chr'] = effect_sizes['chr'].astype(str)
/tmp/ipykernel_13117/2241656604.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effect_sizes['pos_hg38'] = effect_sizes['pos_hg38'].astype(int)
/tmp/ipykernel_13117/2241656604.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

#### Calculo do PRS

In [18]:
calc_vcf_prs('target_cohort_processed.csv', effect_size)

ValueError: 'POS' is not in list